# Baseline network

This notebook trains the baseline network with exact same architecture as the one in paper [Recognition of handwritten Latin characters with diacritics using CNN](https://journals.pan.pl/dlibra/publication/136210/edition/119099/content/bulletin-of-the-polish-academy-of-sciences-technical-sciences-recognition-of-handwritten-latin-characters-with-diacritics-using-cnn-lukasik-edyta-charytanowicz-malgorzata-milosz-marek-tokovarov-michail-kaczorowska-monika-czerwinski-dariusz-zientarski-tomasz-2021-69-no-1?language=en)

Model architecture description:

"The architecture of the concrete CNN is shown in Fig. 2.
The input is a 32x32 binarized matrix. The input is then prop-
agated through 12 adaptable layers. First come two convolu-
tional layers having 32 filters with the size of 3x3 and stride 1.
Secondly, the output of the convolutional layer is fed to the
ReLU function. The output is down-sampled using a max-pool-
ing operation with a 2x2 stride. Next, the dropout technique is
used with the coefficient 0.25. The four operations (two con-
volutions, nonlinearity, max-pooling, and dropout) are repeated,
using 64 filters for the convolutional layers. The output of the
last layer is then flattened and fed through a fully connected
layer with 256 neurons and ReLU nonlinearities, dropped out
with the 0.25 coefficient, and a final output layer is fully con-
nected with a Softmax activation function. The Adam optimizer
and the cross-entropy loss function were used in the network. 
The output is a probability distribution over 89 classes."




After training, model is serialized and uploaded to W&B project.

In [ ]:
import wandb
import tensorflow as tf
import numpy as np
import pathlib
import shutil
from typing import List

def load_data(run, artifact_name = f"phcd_paper_splits_tfds") -> List[tf.data.Dataset]:
    """
    Downloads datasets from a wandb artifact and loads them into a list of tf.data.Datasets.
    """

    artifact = run.use_artifact(f"master-thesis/{artifact_name}:latest")
    artifact_dir = pathlib.Path(
        f"./artifacts/{artifact.name.replace(':', '-')}"
    ).resolve()
    if not artifact_dir.exists():
        artifact_dir = artifact.download()
        artifact_dir = pathlib.Path(artifact_dir).resolve()

    # if tf.__version__ minor is less than 10, use
    # tf.data.experimental.load instead of tf.data.Dataset.load

    if int(tf.__version__.split(".")[1]) < 10:
        load_function = tf.data.experimental.load
    else:
        load_function = tf.data.Dataset.load
    
    output_list = []
    for split in ["train", "test", "val"]:
        ds = load_function(str(artifact_dir / split), compression="GZIP")
        output_list.append(ds)
    
    return output_list

def get_number_of_classes(ds: tf.data.Dataset) -> int:
    """
    Returns the number of classes in a dataset.
    """
    labels_iterator= ds.map(lambda x, y: y).as_numpy_iterator()
    labels = np.concatenate(list(labels_iterator))
    return len(np.unique(labels))

def preprocess_dataset(ds: tf.data.Dataset, batch_size: int, cache: bool = True) -> tf.data.Dataset:
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))  # normalize
    ds = ds.unbatch().batch(batch_size)
    if cache:
        ds = ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

def calculate_model_size_on_disk(path: str) -> int:
    return pathlib.Path(path).stat().st_size    

def calculate_model_num_parameters(model: tf.keras.Model) -> int:
    return model.count_params()

def calculate_model_flops(summary) -> float:
    # from run.summary get GFLOPs or GFLOPS whichever is available
    print(summary.keys())
    if "GFLOPs" in summary.keys():
        return summary.get("GFLOPs")
    elif "GFLOPS" in summary.keys():
        return summary.get("GFLOPS")

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Available devices: ", tf.config.list_physical_devices())

In [ ]:
def train(model_name, defaults, artifact_name="phcd_paper_splits_tfds"):
    with wandb.init(project="master-thesis", job_type="training", name=model_name, config=defaults, tags=[artifact_name]) as run:
        # hyperparameters
        epochs = wandb.config.epochs
        bs = wandb.config.batch_size

        ds_train, ds_test, ds_val = load_data(run, artifact_name=artifact_name)

        num_classes = get_number_of_classes(ds_val)

        ds_train = preprocess_dataset(ds_train, batch_size=bs)
        ds_val = preprocess_dataset(ds_val, batch_size=bs)
        ds_test = preprocess_dataset(ds_test, batch_size=bs, cache=False)

        '''
        The architecture of the concrete CNN is shown in Fig. 2.
        The input is a 32x32 binarized matrix. 
        The input is then propagated through 12 adaptable layers. 
        First come two convolutional layers having 32 filters with the size of 3x3 and stride 1.
        Secondly, the output of the convolutional layer is fed to the
        ReLU function. The output is down-sampled using a max-pool-
        ing operation with a 2x2 stride. Next, the dropout technique is
        used with the coefficient 0.25. The four operations (two con-
        volutions, nonlinearity, max-pooling, and dropout) are repeated,
        using 64 filters for the convolutional layers. The output of the
        last layer is then flattened and fed through a fully connected
        layer with 256 neurons and ReLU nonlinearities, dropped out
        with the 0.25 coefficient, and a final output layer is fully con-
        nected with a Softmax activation function. The Adam optimizer
        and the cross-entropy loss function were used in the network
        [24]. The output is a probability distribution over 89 classes.
        '''

        model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(32, 32, 1)),
                
                # 2 Convolutional layers with 32 filters, 3x3 size, and stride 1
                tf.keras.layers.Conv2D(32, (3, 3), strides=1, activation='relu'),
                tf.keras.layers.Conv2D(32, (3, 3), strides=1, activation='relu'),
                
                # Max-pooling operation with 2x2 stride
                tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
                # Dropout with coefficient 0.25
                tf.keras.layers.Dropout(0.25),
                
                # Repeat above 4 operations using 64 filters
                tf.keras.layers.Conv2D(64, (3, 3), strides=1, activation='relu'),
                tf.keras.layers.Conv2D(64, (3, 3), strides=1, activation='relu'),
                tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
                tf.keras.layers.Dropout(0.25),
                
                # Flatten the output and feed through fully connected layer with 256 neurons
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(256, activation='relu'),

                # Dropout with coefficient 0.25
                tf.keras.layers.Dropout(0.25),
                tf.keras.layers.Dense(num_classes, activation="softmax"),
            ]
        )

        model.compile(
            optimizer="adam",
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=["accuracy"],
        )

        # save the best model
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=f"./artifacts/{model_name}.h5",
            save_weights_only=False,
            monitor="val_accuracy",
            mode="max",
            save_best_only=True,
        )

        wandb_callback = wandb.keras.WandbCallback(
            save_model=False,
            compute_flops=True,
        )

        history = model.fit(
            ds_train,
            epochs=epochs,
            validation_data=ds_val,
            callbacks=[wandb_callback, checkpoint_callback],
        )

        # calculate model size on disk, flops and number of parameters

        flops = calculate_model_flops(wandb.run.summary)
        disk_size = calculate_model_size_on_disk(f"./artifacts/{model_name}.h5")
        num_parameters = calculate_model_num_parameters(model)

        # evaluate model on ds_test and log to wandb
        test_loss, test_acc = model.evaluate(ds_test)

        wandb.log({
            "test loss": test_loss, 
            "test accuracy": test_acc, 
            "number of parameters": num_parameters,
            "disk size": disk_size, 
            "model flops": flops
            })
            
        # save artifact to wandb
        artifact = wandb.Artifact(
            name=model_name,
            type="model"
        )

        # save best model to artifact
        artifact.add_file(f"./artifacts/{model_name}.h5")
        run.log_artifact(artifact)
        run.finish()

In [ ]:
defaults = dict(
    batch_size=32*2,
    epochs=100,    
    optimizer="adam"
)

train("baseline", defaults, artifact_name="uppercase_splits_tfds")